# 2020 US Presidential Elections Winner Simulator
This project uses state-by-state predictions from [PredictIt](https://www.predictit.org/) to simulate many hypothetical elections to create an overall prediction: *how often Joe Biden will win the US election*.  

If you are unfamiliar with Predictit, it is a financial prediction market where traders can exchange shares based on various political events around the world.  The value of these shares is determined by what traders believe is the likelihood of a particular event.

## Imports

In [ ]:
import csv
import io
import json
import numpy
import urllib.request
import re

We need two sets of data: the number of electoral votes allocated to each congressional district, and the market data from PredictIt.

## Download Electoral Votes Data
Get the number of electoral votes assigned to each congressional district.

In [ ]:
url_open = urllib.request.urlopen("https://raw.githubusercontent.com/peterhhchan/us-elections-2020/master/states.csv")
reader = csv.reader(io.TextIOWrapper(url_open, encoding = 'utf-8'), delimiter=',')

electoral_votes = {}
for v,n,a in reader:
    electoral_votes[a]=int(v)
    
print (electoral_votes)

## Download Market Data
Grab the latest share prices on PredictIt. Prices will range from 1 cents to 99 cents (0.01 - 0.99). Roughly, a price of 1 cent implies Biden has a 1% chance of winning that state, and a price of 99 cents implies Biden has a 99% chance of winning that state.

In [ ]:
prices = {}
markets = json.loads(urllib.request.urlopen("https://www.predictit.org/api/marketdata/all/").read())['markets']
for m in markets:
    sn = m['shortName']
    match = re.search("^Which party will win ([A-Z]{2})( in)? 2020\?$", sn)
    if match:
        for c in m['contracts']:
            if c['name'] == 'Democratic':
                prices[match[1]] = c['lastTradePrice']

print (prices)

## Calculate the Implied Probability
PredictIt charges a 10% fee on profits, the savvy bettor understands that in order to break-even on a 50-cent contract, the contract must win more than 52.6% of the time. Here we convert the price to its implied probability. 

In [ ]:
win_prob = {}
for s, price in prices.items():
    p = float (price)
    ## predictit charges a 10% fee on profits
    ## the net profit on a 50 cent contract is 45 cents
    ## the bettor must win 52.63%+ of the time to breakeven
    win_prob[s] = p / ((1 - p) * 0.9 + p)
print (win_prob)

## Adjust the Probabilities 
If you think the markets are incorrect, this section is where you could adjust the predictions.

Here we apply a global bias. If we think the markets are over-valueing Biden's chances by 25%, we would set our bias to `0.8`.

In [ ]:
bias = 1
def apply_bias():
    for s, prob in win_prob.items():
        win_prob [s] = prob * bias
apply_bias()

Next, we can apply smooth out our simulations by applying some assumptions to our predictions. 

 * If the market prediction is above `win_threshold`, Biden will always win that district
 * If the market prediction is below `lose_threshold`, Biden will always lose that district.

An interesting set of parameters to use is to set `win_threshold` to *0.8*, and `lose_threshold` to *0.5*.

In [ ]:
win_threshold  = 0.95
lose_threshold = 0.05

In [ ]:
def apply_thresholds():
    for s, prob in win_prob.items():
        if prob > win_threshold:   
            win_prob [s] = 1.0
        elif prob < lose_threshold: 
            win_prob [s] = 0

apply_thresholds() 
print (win_prob)

Alternatively, you can simply update a state's probability via: `win_prob[FL] = 0.5` which means Biden has a 50% chance of winning Florida.

In [ ]:
## Override Democrat's chances of winning
# win_prob['FL'] = 0.5
# win_prob['AZ'] = 0.5
# win_prob['NC'] = 0.5
# win_prob['WI'] = 0.5
# win_prob['MI'] = 0.5
# win_prob['PA'] = 0.5
# win_prob['MN'] = 0.5

## Prepare the Data
1. Arrange the data alphabetically

In [ ]:
win_prob_sorted = sorted(win_prob.items())
print (win_prob_sorted)

2. Create a `numpy` array of the win probabilities

In [ ]:
ps_sorted = numpy.fromiter(dict(win_prob_sorted).values(), dtype=float)
print (ps_sorted)

3. Create a `numpy` array of the electoral votes

In [ ]:
vs_sorted = numpy.fromiter ((v for _,v in sorted(electoral_votes.items())), int)    
print (vs_sorted)

Combining steps 1-3 into a function

In [ ]:
def update_arrays():
    win_prob_sorted = sorted(win_prob.items())
    ps_sorted = numpy.fromiter(dict(win_prob_sorted).values(), dtype=float)
    vs_sorted = numpy.fromiter ((v for _,v in sorted(electoral_votes.items())), int)    

## Create and Run the Simulation
1. Generate a bunch of random numbers, and compare the numbers to the implied probabilities
2. If the implied probability is greater than the random number, give Biden the number of votes that corresponds to that state
3. Sum up all the votes and see if that number is bigger than 269
4. Run steps 1-3 repeatedly and calculate how often Biden wins

In [ ]:
def simulate(n):
    update_arrays()
    
    rs = numpy.random.rand(n, len(win_prob_sorted))
    sims_won = 0
    for r in rs:
        wins = numpy.greater(ps_sorted , r)
        total_votes_won = numpy.sum(numpy.multiply(wins, vs_sorted))
        if total_votes_won > 269: ## Tie breaks go to republicans
            sims_won+=1

    return sims_won / n

Run the simulation!

In [ ]:
simulate (100000)

## Limitations
1. The biggest problem with this simulator is that it treats the individual results as independent events, which they are not.  

2. Nebraska and Maine uses the congressional district method to assign their votes. However, their electoral votes are assigned using the winner-take-all system in our simulations.

## Further Reading
https://www.270towin.com/